# <span style="color:orange">Nettoyer des jeux de données pour obtenir une liste de DOI des publications de l'Université de Toulon : Web of Science, Pubmed, HAL, données des APC, Lens.org</span>

Ce premier notebook sert à nettoyer les différents fichiers obtenus après téléchargement sur le WoS, Pubmed, HAL, les données d'APC et Lens.org. Pour savoir quelques requêtes ont été utilisées pour l'Université de Toulon, consulter le fichier intitulé "requetes_bdd" dans le dossier. Quelques consignes sont à respecter pour que tout fonctionne :

- Pour le WoS, il suffit de procéder à un téléchargement simple "Fast 5000". Le fichier obtenu, en texte, est illisible et c'est normal, il n'y a rien à changer. Nommer le fichier "wos_toulon_2016", puis "wos_toulon_2017"... Ce fichier n'apparaît pas dans le dossier téléchargé depuis Gitlab car les données du Web of Science étant propriétaires, il n'était pas possible de les diffuser librement.
- Pour Pubmed, le téléchargement donne un fichier CSV très peu classé, c'est normal, il n'y a rien à changer. Nommer le fichier "pubmed_toulon_2016", puis "pubmed_toulon_2017"...
- Pour les autres sources de données, on obtient directement une liste de DOI, mais **il faut s'assurer que la colonne s'appelle bien "doi" en minuscules et qu'il n'y a pas de ligne vide**

Il faut télécharger année par année, et toujours nommer les fichiers de la même manière.  <span style="color:red"> **Il est vital de garder l'organisation ici présente (Data > raw > dossier par année) pour que le code fonctionne.**</span>

<span style="color:red">Si l'on ne dispose pas de certaines données (par exemple, l'établissement n'a pas de données sur les APC ou n'utilise pas le Web of Science), il ne faut pas exécuter les parties de code liées à ces outils. Si l'on ne dispose pas d'extractions du Web of Science, on n'exécute pas toute la partie "Nettoyer les données issues du Web of Science".

Il faut remplacer "toulon" par le nom de l'établissement directement dans le code ci-dessous. Vous pouvez faire ctrl-f pour modifier toutes les occurrences d'un coup.**</span>

Commencer par exécuter les lignes ci-dessous : cliquer sur la ligne puis ensuite sur le bouton "play" de la bare d'outils.

In [1]:
column_name = "doi"

In [2]:
import pandas

In [3]:
import csv

## <span style="color:orange">Nettoyer les données issues du Web of Science</span>

### Comprendre comment est structuré le fichier

On ouvre le premier fichier puis on utilise la méthode de liste **split** pour voir une ligne (la 4ème, par exemple).

In [4]:
with open("Data/raw/2016/wos_toulon_2016.txt") as f:
    wos_string = f.read()

La ligne ci-dessous permet de voir à quoi ressemble la 5ème ligne du fichier (en Python, la numérotation commence à zéro).

In [5]:
wos_string.split("\n") [4]

'J\tManzanera, Antoine; Thanh Phuong Nguyen; Xu, Xiaolei\t\t\t\t\t\t\t\tLine and circle detection using dense one-to-one Hough transforms on greyscale images\t\t\t\t\t\t\t\tEURASIP JOURNAL ON IMAGE AND VIDEO PROCESSING\t\t\t\t\t\t\t\t\t\t46\t10.1186/s13640-016-0149-y\t\t\tDEC 12 2016\t2016\tBy estimating the first-order (direction) and second-order (curvature) derivatives in an image, the parameters of a line or circle passing through a point may be uniquely defined in most cases. This allows to compute a one-to-one Hough transform, every point in the image space voting for one unique point in the parameter space. Moreover, those parameters can be directly estimated on the greyscale image without the need to calculate the contour and without reducing the spatial support of the Hough transform, i.e. densely on the whole image. The general framework using multiscale derivatives is presented, and the one-to-one Hough dense transforms for detecting lines and circles are evaluated and compa

### Définir une fonction

Définition de la fonction **read_wos** : l'année est en argument. On tape la 1ère année concernée (2016) pour que cela utilise les données de 2016 en premier. Dans cette fonction, les fichiers de toutes les années (de 2016 à 2021 exclue) seront interrogés. 

On crée une liste vide (wos_per_year) dans laquelle on stockera après les données de toutes les années grâce à la boucle **for**.  <span style="color:red">**Pour l'instant, on n'interroge que jusqu'en 2020. Il faudra mettre 2022 à la place quand les données de 2021 seront ajoutées dans le dossier "raw", et ainsi de suite.**</span>

La liste vide wos_per_year est remplie à chaque tour de boucle grâce à la méthode **append**. Pour avoir la liste de toutes les données sans tri par année, on concatène les fichiers grâce à **pandas.concat**.

In [6]:
def read_wos(year):
    wos_per_year = []
    for year in range (2016, 2021):
       wos_df = pandas.read_csv("Data/raw/{}/wos_toulon_{}.txt".format(str(year), str(year)), sep="\t",
                 quoting=csv.QUOTE_NONE, index_col=False, usecols=['DI'])
       wos_per_year.append(wos_df)

    full_wos = pandas.concat(wos_per_year)
    
    resultat = full_wos.rename (columns={'DI':column_name})
           
    return resultat

In [7]:
wos_df = read_wos(2016)

In [8]:
wos_df

,doi
0,10.1103/PhysRevD.94.123520
1,10.1186/s13068-016-0681-0
2,10.1186/s13068-016-0678-8
3,10.1186/s13640-016-0149-y
4,10.1088/1751-8113/49/49/495202
...,...
328,10.1016/j.scitotenv.2019.134067
329,10.1016/j.ceramint.2019.09.015
330,10.3934/dcds.2020015
331,10.1016/j.astropartphys.2019.06.003


Vérifier que le chiffre total obtenu ici correspond bien au chiffre trouvé dans le Web of Science pour toutes les années. Ne pas hésiter à refaire les extractions d'années précédentes si les chiffres ne correspondent pas : il peut y avoir de nouvelles publications indexées dans le Web of Science.

### Identifier les cellules qui n'ont pas de DOI

Trouver le nombre de lignes qui n'ont pas de DOI : utiliser la méthode **is not a number** (isna).

In [9]:
def wos_sans_doi():
    wos_df.isna().sum()
    return wos_df.isna().sum()

On met [0] pour demander le 1er élément de la liste

In [10]:
wos_sans_doi()[0]

228

Trouver le nombre de lignes qui ont un DOI : utiliser la méthode **match** sur le string.

In [11]:
def wos_avec_doi():
    wos_df[column_name].str.match("10.").sum()
    return wos_df[column_name].str.match("10.").sum()

In [12]:
wos_avec_doi()

1652

Vérifier le nombre de lignes total :

In [13]:
len(wos_df)

1880

Vérifier qu'on a bien tout récupéré :

In [14]:
wos_sans_doi() + wos_avec_doi()

doi    1880
dtype: int64

Méthode de vérification de Python : **assert**. Si cela ne renvoie rien, c'est que les deux valeurs comparées sont identiques.

In [15]:
expected = len(wos_df[column_name])
actual = wos_sans_doi()[0] + wos_avec_doi()

assert(expected == actual)

 <span style="color:green"> **Les données du WoS sont à présent nettoyées et stockées dans la variable intitulée "wos_df".** </span>

## <span style="color:orange">Nettoyer les données issues de Pubmed</span>

In [16]:
with open("Data/raw/2016/pubmed_toulon_2016.csv", encoding='utf-8') as f:
    pubmed_string = f.read()

In [17]:
pubmed_string.split("\n") [1]

'"27183120","Effects of Footwear and Fatigue on Running Economy and Biomechanics in Trail Runners","Vercruyssen F, Tartaruga M, Horvais N, Brisswalter J.","Med Sci Sports Exerc. 2016 Oct;48(10):1976-84. doi: 10.1249/MSS.0000000000000981.","Vercruyssen F","Med Sci Sports Exerc","2016","2016/05/17","","","10.1249/MSS.0000000000000981"'

In [18]:
pubmed_df = pandas.read_csv("Data/raw/2020/pubmed_toulon_2020.csv", encoding='utf-8')

In [19]:
print(pubmed_df.columns)

Index(['PMID', 'Title', 'Authors', 'Citation', 'First Author', 'Journal/Book',
       'Publication Year', 'Create Date', 'PMCID', 'NIHMS ID', 'DOI'],
      dtype='object')


Il y a bien une colonne qui s'appelle "DOI", elle va être interrogée pour trouver directement les DOI. <span style="color:red">**Pour l'instant, on n'interroge que jusqu'en 2020. Il faudra mettre 2022 à la place quand les données de 2021 seront ajoutées dans le dossier "raw", et ainsi de suite.**</span>


In [20]:
def read_pubmed(year):
    pubmed_per_year = []
    for year in range (2016, 2021):
        pubmed_df = pandas.read_csv("Data/raw/{}/pubmed_toulon_{}.csv".format(str(year), str(year))
                                    ,sep=",", encoding='utf-8')
        pubmed_per_year.append(pubmed_df)
    
    full_pubmed = pandas.concat(pubmed_per_year)
           
    resultat = pandas.DataFrame({column_name: full_pubmed["DOI"]})                      
    return resultat

In [21]:
pubmed_df = read_pubmed(2016)

In [22]:
pubmed_df

,doi
0,10.1249/MSS.0000000000000981
1,10.4187/respcare.04821
2,10.1113/JP271167
3,10.1093/femsle/fnv216
4,10.1177/0031512516656817
...,...
147,10.1103/PhysRevLett.125.232003
148,10.1152/japplphysiol.00259.2020
149,10.1016/j.marpolbul.2019.110870
150,10.3390/bios10080093


### Vérifier qu'il ne manque pas de DOI

In [23]:
def pubmed_sans_doi():
    pubmed_df.isna().sum()
    return pubmed_df.isna().sum()

In [24]:
pubmed_sans_doi()

doi    2
dtype: int64

In [25]:
def pubmed_avec_doi():
    pubmed_df[column_name].str.match("10.").sum()
    return pubmed_df[column_name].str.match("10.").sum()

In [26]:
pubmed_avec_doi()

496

In [27]:
expected = len(pubmed_df)
actual = pubmed_sans_doi()[0] + pubmed_avec_doi()

assert(expected == actual)

 <span style="color:green"> **Les données de Pubmed sont à présent nettoyées et stockées dans la variable intitulée "pubmed_df".** </span>

## <span style="color:orange">Ajouter les données de HAL</span>

Grâce à ExtrHAL, il est très simple de récupérer une liste de DOI. Il peut être intéressant de refaire des extractions d'une année sur l'autre quand des campagnes de dépôt massif sont organisées, par exemple.

Ouvrir le fichier et concaténer les différentes années (mettre "2022" dans le code quand les données de 2021 seront injectées) :

In [28]:
def read_hal(year):
    hal_per_year = []
    for year in range (2016, 2021):
        hal_df = pandas.read_csv("Data/raw/{}/hal_toulon_{}.csv".format(str(year), str(year)), sep="\t", encoding='latin-1')
        hal_per_year.append(hal_df)
    
    
    full_hal = pandas.concat(hal_per_year)
                     
    return full_hal

In [29]:
hal_df = read_hal(2016)

In [30]:
hal_df

,doi
0,10.1088/1674-1137/40/1/011001
1,10.1016/j.jct.2015.08.019
2,10.1016/j.compositesb.2016.02.034
3,10.1016/j.actamat.2016.03.007
4,10.1016/j.tet.2016.01.038
...,...
593,10.1002/mbo3.1031
594,10.1016/j.oceaneng.2020.107190
595,10.17791/jcs.2020.21.2.193
596,10.1007/s10853-020-04856-4


 <span style="color:green"> **Les données de HAL sont à présent nettoyées et stockées dans la variable intitulée "hal_df".** </span>

## <span style="color:orange">Ajouter les données issues des paiements d'APC</span>

Ces données proviennent des fichiers des bibliothèques de Toulon, établis pour le Consortium Couperin pour le suivi des APC. Il faut demander les fichiers à la personne qui en a la charge.

Ici, les années 2016-2020 sont présentes dans le jeu de données. Il n'y a pas de chiffres pour les années ultérieures pour le moment. Comme nous n'avions pas les données de 2016, nous les avons remplacer par celles de 2015 dans le fichier "apc_toulon_2016". 
Il faut donc écrire 2016-2021 pour que les années de 2016 à 2020 incluse soient comptabilisées.

 <span style="color:red"> **Si votre établissement ne dispose pas de ces données, il suffira d'enlever la variable "apc_df" qui est utilisée plus bas, sur la ligne "all_toulon".** </span>

In [31]:
def read_apc(year):
    apc_per_year = []
    for year in range (2016, 2021):
        apc_df = pandas.read_csv("Data/raw/{}/apc_toulon_{}.csv".format(str(year), str(year)), sep="\t", encoding='latin-1')
        apc_per_year.append(apc_df)
    
    
    full_apc = pandas.concat(apc_per_year)
                     
    return full_apc

In [32]:
apc_df = read_apc(2016)

In [33]:
apc_df

,doi
0,10.1155/2015/245378
1,10.1175/JPO-D-14-0247.1
2,10.1371/journal.pone.0136497
3,10.1371/journal.pcbi.1004579
4,10.1186/s12879-016-2003-3
5,10.1186/1944-3277-10-12
0,10.5194/angeo-35-423-2017
1,10.5194/bg-14-2429-2017
2,10.3389/fmicb.2017.01399 OU 10.3389/fmicb.201...
3,10.1371/journal.pone.0185741


 <span style="color:green"> **Les données d'APC sont à présent nettoyées et stockées dans la variable intitulée "apc_df".** </span>

## <span style="color:orange">Ajouter les données issues de Lens.org</span>

Ces données proviennent de la base de données Lens.org https://www.lens.org/

Attention, l'export direct de la colonne "DOI" produit un fichier dont la colonne s'appelle "DOI" : il faut la renommer en "doi". Il faut également enlever les lignes vides qui pourraient éventuellement être présentes dans ce tableau. Voici les commandes Excel : sélectionner la colonne "doi". Dans le menu "rechercher et sélectionner", cliquer sur "Atteindre", puis "Cellules". Cocher "cellules vides" et valider. Puis dans la barre du haut, cliquer sur "Supprimer" et  "Supprimer les cellules…", "lignes entières".

In [34]:
def read_lens(year):
    lens_per_year = []
    for year in range (2016, 2021):
        lens_df = pandas.read_csv("Data/raw/{}/lens_toulon_{}.csv".format(str(year), str(year)), sep="\t", encoding='latin-1')
        lens_per_year.append(lens_df)
    
    
    full_lens = pandas.concat(lens_per_year)
                     
    return full_lens

In [35]:
lens_df = read_lens(2016)

In [36]:
lens_df

,doi;,doi
0,10.1109/cca.2016.7587905;,NaN
1,10.1007/s13042-016-0625-9;,NaN
2,10.1002/2016jc011961;,NaN
3,10.1016/j.syapm.2019.02.005; 10.1016/j.syapm.2...,NaN
4,10.3389/fmicb.2016.01303;,NaN
...,...,...
128,NaN,10.1016/j.nanoso.2019.100422
129,NaN,10.1109/ic_aset49463.2020.9318261
130,NaN,10.1557/mrc.2020.29
131,NaN,10.3166/rfg.2020.00488


 <span style="color:green"> **Les données de Lens.org sont à présent nettoyées et stockées dans la variable intitulée "lens_df".** </span>

## <span style="color:orange">Rassembler les 5 listes de DOI (WoS, PubMed, HAL, APC) et enlever les doublons</span>

Si un établissement utilise Scopus à la place du Web of Science, remplacer "wos_df" par "scopus_df".

### Concaténer les 5 listes de données

 <span style="color:red"> **Pour adapter ce code à votre établissement, vous pouvez changer le nom de la variable "all_toulon". Mais attention dans ce cas à bien modifier le nom de la variable dans tout le reste du code.** </span>

In [37]:
all_toulon = pandas.concat([wos_df, pubmed_df, hal_df, apc_df, lens_df])

### Enlever les lignes sans DOI

In [38]:
import numpy

In [39]:
mask_doi = all_toulon[column_name].notna()
mask_doi

0      True
1      True
2      True
3      True
4      True
       ... 
128    True
129    True
130    True
131    True
132    True
Name: doi, Length: 6246, dtype: bool

Le résultat correspond à la concaténation de toutes les lignes des 5 fichiers, y compris celles qui ne comprennent pas de DOI.

In [40]:
doi_toulon_doublons = all_toulon[mask_doi]

In [41]:
doi_toulon_doublons

,doi,doi;
0,10.1103/PhysRevD.94.123520,NaN
1,10.1186/s13068-016-0681-0,NaN
2,10.1186/s13068-016-0678-8,NaN
3,10.1186/s13640-016-0149-y,NaN
4,10.1088/1751-8113/49/49/495202,NaN
...,...,...
128,10.1016/j.nanoso.2019.100422,NaN
129,10.1109/ic_aset49463.2020.9318261,NaN
130,10.1557/mrc.2020.29,NaN
131,10.3166/rfg.2020.00488,NaN


Nous avons ici la liste de tous les DOI recensés dans nos 5 sources d'information. Mais attention, il peut y avoir des doublons : les DOI peuvent notamment être écrits en minuscules comme en majuscules.

### Enlever les doublons

Avant d'enlever les doublons, il faut normaliser les DOI et tous les passer en minuscules.

In [42]:
doi_toulon_doublons_minuscule = doi_toulon_doublons[column_name].str.lower()

In [43]:
doi_toulon_doublons_df = pandas.DataFrame({column_name : doi_toulon_doublons_minuscule}) 

Après avoir passé l'ensemble de la DataFrame en minuscules, on peut enlever les doublons.

In [44]:
doi_toulon_final = doi_toulon_doublons_df.drop_duplicates()

Réindexer la DataFrame pour que le nom des lignes soit propre :

In [45]:
doi_toulon_final.reset_index(drop=True, inplace=True)

In [46]:
doi_toulon_final

,doi
0,10.1103/physrevd.94.123520
1,10.1186/s13068-016-0681-0
2,10.1186/s13068-016-0678-8
3,10.1186/s13640-016-0149-y
4,10.1088/1751-8113/49/49/495202
...,...
3769,10.1109/ijcnn48605.2020.9207145
3770,10.1051/epjconf/202022504009
3771,10.32735/s0718-2201202000050798
3772,10.1109/ic_aset49463.2020.9318261


**On obtient 21 457 DOI pour la période 2016-2020.**  <span style="color:red"> **Ce chiffre est à actualiser chaque année : dans le code, cela se fera automatiquement.** </span>

### Créer dans le dossier "outputs" un fichier CSV avec la liste des DOI

De même que précédemment, chaque établissement peut utiliser un nom différent : il faut simplement penser à remplacer "toulon" par le nom de l'établissement à chaque fois qu'il apparaît dans le code et toujours l'écrire de la même façon.

In [47]:
doi_toulon_final.to_csv("Data\outputs\doi_toulon.csv",index=False)

Pour obtenir cette liste dans un format Excel classique, saisir le code suivant.

In [48]:
doi_toulon_final.to_csv("Data\outputs\doi_toulon.xls",index=False)